<a href="https://colab.research.google.com/github/iLikeKatz/MyLeaning_ML/blob/main/PytorchFullCourse/Models/Multi_Cassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

In [41]:
device = "cuda" if torch.cuda.is_available() else "cpu"
n_samples = 1000
n_features  = 2
n_classes = 4
x, y = make_blobs(n_samples =n_samples, n_features=2, centers=n_classes, random_state=42, cluster_std=1.5)
x = torch.from_numpy(x).to(device).type(torch.float)
y = torch.from_numpy(y).to(device).type(torch.long)

In [42]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.2)
x.shape, y.shape

(torch.Size([1000, 2]), torch.Size([1000]))

In [ ]:
class Multi(nn.Module):
  def __init__(self, in_features, out_features, hidden=8):
    super().__init__()
    self.stack_layers = nn.Sequential(
        nn.Linear(in_features, hidden),
        nn.ReLU(),
        nn.Linear(hidden, hidden),
        nn.ReLU(),
        nn.Linear(hidden, out_features)
    )

  def forward(self, x):
    x = self.stack_layers(x)
    return x

model = Multi(in_features=n_features, out_features=n_classes, hidden=12).to(device)
model.state_dict()

In [44]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs = 2000


In [45]:
for epoch in range(epochs):
  model.train()
  y_logits= model(x_train) #the output will be the values of each columns
  y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) #so we need to use softmax to convert each values of columns to probabilities and then we will use .argmax in dim1 to show the most probability values in each rows

  loss = loss_fn(y_logits, y_train)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  model.eval()
  with torch.inference_mode():
    y_logits_ = model(x_test)
    y_pred_  = torch.softmax(y_logits_, dim=1).argmax(dim=1)
    loss_test = loss_fn(y_logits_, y_pred_)
    if epoch % 100 == 0:
      print(f"epoch : {epoch}, loss : {loss}, loss_test : {loss_test}")

epoch : 0, loss : 1.1694415807724, loss_test : 0.965263843536377
epoch : 100, loss : 0.021410437300801277, loss_test : 0.007150260265916586
epoch : 200, loss : 0.018543459475040436, loss_test : 0.0072233593091368675
epoch : 300, loss : 0.017083974555134773, loss_test : 0.005195074714720249
epoch : 400, loss : 0.016519244760274887, loss_test : 0.004064951557666063
epoch : 500, loss : 0.016083573922514915, loss_test : 0.003563585923984647
epoch : 600, loss : 0.015493012964725494, loss_test : 0.0032390370033681393
epoch : 700, loss : 0.014826472848653793, loss_test : 0.0028812831733375788
epoch : 800, loss : 0.01416113879531622, loss_test : 0.0026906474959105253
epoch : 900, loss : 0.013566549867391586, loss_test : 0.002675306284800172
epoch : 1000, loss : 0.013277177698910236, loss_test : 0.0021135404240339994
epoch : 1100, loss : 0.012862886302173138, loss_test : 0.001990766730159521
epoch : 1200, loss : 0.013253421522676945, loss_test : 0.0014263041084632277
epoch : 1300, loss : 0.0124

In [46]:
from sklearn.metrics import accuracy_score as acc
model.eval()
with torch.inference_mode():
    y_logits__ = model(x_test)
    y_pred__  = torch.softmax(y_logits__, dim=1).argmax(dim=1)
    loss_test = loss_fn(y_logits__, y_pred__)
acc(y_pred__, y_test) * 100

100.0